<a href="https://colab.research.google.com/github/aguiarsm/WebScraping_TripAdv/blob/main/2_Raspando.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [ ]:
#!/usr/bin/env python
# coding: utf-8


import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np


def raspa_pagina(soup, tag_html, atributo, match_atributo, regex):
    """Função que irá extrair os dados de interesse"""

    data_soup = soup.find_all(tag_html, attrs={atributo, match_atributo})
    x = []

    for prov in (data_soup):
        _ = re.findall(regex, str(prov))
        if _ != []:
            if regex == 'rating bubble_(..)':
                x.append(int(_[0]) // 10)
            else:
                x.append(_[0])
        else:
            x.append("ERRO!!!")
    return x


#Hotel em que será iniciada e terminada a raspagem (O ranking do)
hotel_inicio =  1#@param {type:"integer"}
hotel_fim =  25#@param {type:"integer"}
ano_limite = 2021 #@param {type:"slider", min:2010, max:2022, step:1}
limite_de_paginas = 2 #@param {type:"number"}


#configurando Selenium
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

#Dados de suporte
_este_ano = ['Hoje', 'Ontem', 'jan.', 'fev.', 'mar.', 'abr.', 'mai.', 'jun.', 'jul.', 'ago.', 'set.', 'out.', 'nov.', 'dez.']
_ano = [2022] * 5


# link da lista de ganhadores
url = 'https://www.tripadvisor.com.br/TravelersChoice-Hotels-cTop-g294280'

driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

#Dados da lista de hoteis
all_hotel = soup.find_all("div", attrs={"class", "winnerName"})
    #Raspando informações básicas dos hotéis
identifica_hoteis = []
for pos, prov in enumerate(all_hotel):
    endereco = prov.find_all("a")[0]['href']
    cidade = re.search(f'target="_blank">(.*), Brasil<\/a>', str(prov.find_all("a")[1])).group(1)
    nome = re.search(r'target="_blank">[0-9\. ]*(.*)<\/a>', str(prov.find_all("a")[0])).group(1)
    identifica_hoteis.append([nome, cidade, "https://www.tripadvisor.com.br" + endereco])

df_lista = pd.DataFrame(identifica_hoteis, columns=['nome_hotel', 'cidade', 'link'])
links_hoteis = df_lista['link'].to_list()  # Lista com link de todos os hoteis

#DataFrame onde serão armazenados os dados
df_hotel = pd.DataFrame(columns=['ranking', 'nome_hotel', 'nota', 'titulo', 'texto', 'data', 'ano'])

#Esse for irá iterar sobre cada Hotel
for Seleciona_hotel in range(hotel_inicio - 1, hotel_fim - 1):
    pagina = 0
    
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get(links_hoteis[Seleciona_hotel]) # Pega o link de um dado Hotel
    soup_hotel = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    nome_hotel = re.findall(r'HEADING">(.*)<\/h1>', str(soup_hotel.find_all("h1", attrs={"class", "QdLfr b d Pn"}))) * 5
    prox_page = soup_hotel.find_all("a", attrs={"class", "ui_button nav next primary "})
    link_split = prox_page[0]['href'].split('-or5-')

    #Criando lista de páginas para um mesmo hotel
    link_hotel_paginas = []
    for num_coment_inf in range(0, limite_de_paginas * 5, 5):
        if num_coment_inf == 0:
            link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1])
        else:
            link_hotel_paginas.append(
        "https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1])


    #Raspando as avaliações
    while _ano[-1] >= ano_limite and pagina < limite_de_paginas:
        driver = webdriver.Chrome('chromedriver',options=chrome_options)
        driver.get(link_hotel_paginas[pagina])
        soup_hotel_paginas = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()

        #Extraindoos dados
        _nota = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)')  # NOTAS
        _texto = raspa_pagina(soup_hotel_paginas, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>')  # TEXTO
        _titulo = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'KgQgP MC _S b S6 H5 _a',
                               '<span><span>(.*)<\/span><\/span>')  # TITULO
        _data = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'cRVSd',
                             'escreveu uma avaliação (.*)<\/span><\/div>')  # DATA
        _ano = []        
        
        for prov in _data:
           if prov.split()[-1] in _este_ano:
               _ano.append(int(2022))
           else:
               _ano.append(int(prov.split()[-1]))

        #Escrevendo no dataframe
        df_prov = pd.DataFrame(np.transpose(np.array([[Seleciona_hotel] * 5, nome_hotel, _nota, _titulo, _texto, _data, _ano])), \
                               columns=['ranking', 'nome_hotel', 'nota', 'titulo', 'texto', 'data', 'ano'])
        df_hotel = pd.concat([df_hotel, df_prov], ignore_index=True)

        print(nome_hotel[0], _data[-1])
        pagina += 1

        #Checkpoint de salvamento
        if pagina % 5 == 0:
            df_hotel.to_csv(f'hoteis_de_{hotel_inicio}_{hotel_fim - 1}.csv', index=False)


#df_hotel.to_csv(f'hoteis_de_{hotel_inicio}_{hotel_fim - 1}.csv', index=False)

Hotel Colline de France Ontem
Hotel Colline de France 22 de jul.
Hotel Valle D’Incanto 24 de jul.
Hotel Valle D’Incanto 21 de jul.
Hotel Ritta Höppner 22 de jul.
Hotel Ritta Höppner 15 de jul.
Hotel Refugio da Montanha 18 de jul.
Hotel Refugio da Montanha 9 de jul.
Hotel Estalagem St Hubertus 13 de jul.
Hotel Estalagem St Hubertus jun. de 2022
Hotel Le Renard 18 de jul.
Hotel Le Renard 3 de jul.
Carballo Hotel &amp; Spa abr. de 2022
Carballo Hotel &amp; Spa abr. de 2022
Porto Pacuíba Hotel 14 de jul.
Porto Pacuíba Hotel mai. de 2022
Salinas Maragogi All Inclusive Resort Ontem
Salinas Maragogi All Inclusive Resort 22 de jul.
Casas Brancas Boutique Hotel &amp; Spa jun. de 2022
Casas Brancas Boutique Hotel &amp; Spa mar. de 2022
Hotel Fasano Salvador 2 de jul.
Hotel Fasano Salvador mai. de 2022
Santa Clara Eco Resort 20 de jul.
Santa Clara Eco Resort 10 de jul.
NANNAI Muro Alto 23 de jul.
NANNAI Muro Alto 18 de jul.
Hotel Unique mar. de 2022
Hotel Unique nov. de 2021
Salinas Maceió All In

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
df_hotel #(Fiz só duas páginas por hotel, apenas para visualização)

,ranking,nome_hotel,nota,titulo,texto,data,ano
0,0,Hotel Colline de France,5,Final de Semana Romântico,"Excelente! Local lindíssimo, com charme e requ...",Ontem,2022
1,0,Hotel Colline de France,5,Fantástico o atendimento as acomodações e tudo...,"Espetacular o atendimento, as acomodações, a d...",Ontem,2022
2,0,Hotel Colline de France,5,Não à toa é o MELHOR HOTEL DO MUNDO!,Melhor experiência que já tive em uma viagem!!...,Ontem,2022
3,0,Hotel Colline de France,5,O melhor hotel de Gramado,"Simplesmente encantador, tudo impecável! Aten...",Ontem,2022
4,0,Hotel Colline de France,5,Experiência fantástica,Com certeza esse hotel merece o título o qual ...,Ontem,2022
5,0,Hotel Colline de France,5,Passeio,"Experiência incrível, desde que cheguei já sen...",Ontem,2022
6,0,Hotel Colline de France,5,Espetáculo!,Retornamos ao Colline de France. A cada visita...,24 de jul.,2022
7,0,Hotel Colline de France,5,25 anos de casados,Excelente!!! Melhor hotel da vidaaa!! Adoramo...,22 de jul.,2022
8,0,Hotel Colline de France,5,Melhor hotel do mundo.,"Tudo no hotel é maravilhoso, o atendimento é i...",22 de jul.,2022
9,0,Hotel Colline de France,5,25 anos de casamento,Ficamos muito satisfeito com o Hotel e com o a...,22 de jul.,2022
